<a href="https://colab.research.google.com/github/Leixb/UPC-ADSDB_e2e/blob/master/notebooks/landing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from zipfile import ZipFile
import glob
import os
from datetime import datetime
import json
import hashlib

In [2]:
try:
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)
  is_local = False
except ModuleNotFoundError:
  is_local = True

Mounted at /content/drive


In [6]:
folder_landing = "./landing" if (is_local) else "/content/drive/MyDrive/ADSDB/landing"

folder_temporal = os.path.join(folder_landing, "temporal")
folder_persistent = os.path.join(folder_landing, "persistent")

extract_dir = os.path.join(folder_persistent, "extracted")

In [12]:
def sha256_file(filename):
  with open(filename,"rb") as f:
    return hashlib.sha256(f.read()).hexdigest();

In [19]:
time_format = "%Y/%m/%d %H:%M:%S"

global_metadata = dict()

for zip_file in glob.glob(os.path.join(folder_temporal, "*.zip")):
  print(zip_file)

  filename = os.path.basename(zip_file)

  modification_time = datetime.fromtimestamp(os.path.getmtime(zip_file))
  modification_timestamp = modification_time.timestamp()

  ingestion_time = datetime.now()
  ingestion_timestamp = ingestion_time.timestamp()

  sha256 = sha256_file(zip_file)

  out_dir = os.path.join(extract_dir, f"{filename[:-4]}-{sha256}-{ingestion_timestamp}")
  os.makedirs(out_dir, exist_ok = True)

  print(out_dir)

  with ZipFile(zip_file, 'r') as zipObj:
    zipObj.extractall(out_dir)

  metadata = {
      "filename" : filename,
      "source" : os.path.relpath(zip_file, start = out_dir),
      "sha256" : sha256,

      "modification_timestamp" : modification_timestamp,
      "modification_time" : modification_time.strftime(time_format),

      "ingestion_timestamp" : ingestion_timestamp,
      "ingestion_time" : ingestion_time.strftime(time_format),
  }

  with open(os.path.join(out_dir, "metadata.json"), 'w') as outfile:

    json.dump(metadata, outfile, indent=2, sort_keys=True)

  global_metadata[f"{filename}-{ingestion_timestamp}"] = metadata

glob_meta_file = os.path.join(folder_persistent, "global_metadata.json")

try:
  with open(glob_meta_file, 'r') as f:
    old_meta = json.load(f)
except FileNotFoundError:
  old_meta = dict()

joined_meta = {**old_meta, **global_metadata}

with open(glob_meta_file, 'w') as f:
  old_meta = json.dump(joined_meta, f, indent=2, sort_keys=True)

/content/drive/MyDrive/ADSDB/landing/temporal/morticd07.zip
/content/drive/MyDrive/ADSDB/landing/persistent/extracted/morticd07-22b463f703974c247c37a33133713348800a4cc68163eb37ff7ac139406237b5-1642855877.728462
/content/drive/MyDrive/ADSDB/landing/temporal/morticd10_part3.zip
/content/drive/MyDrive/ADSDB/landing/persistent/extracted/morticd10_part3-1c6ac83e205760a121b515cad8d97733f8cdc19afd0beb0b5d0e45d474786f54-1642855878.072968
/content/drive/MyDrive/ADSDB/landing/temporal/morticd09.zip
/content/drive/MyDrive/ADSDB/landing/persistent/extracted/morticd09-71c96f66c67f0ae8fb0da942e9ba1a0b050b6d60450c58997c49fe6ba16f9b6f-1642855879.053405
/content/drive/MyDrive/ADSDB/landing/temporal/morticd10_part2.zip
/content/drive/MyDrive/ADSDB/landing/persistent/extracted/morticd10_part2-ffb1ddb02ca487f0e074977c486fe89dc61d1180041b53e5be5872f7cf61e0da-1642855880.083453
/content/drive/MyDrive/ADSDB/landing/temporal/morticd08.zip
/content/drive/MyDrive/ADSDB/landing/persistent/extracted/morticd08-9366